<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Examples" data-toc-modified-id="Examples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Examples</a></span></li></ul></div>

# Chain of Responsibility

## Intro

This pattern focuses on decoupling the sender of a request to its receiver by giving it more than one object a chance to handle the request during its life. The chain receiving the object passes the request until the object that handles it does so.

This pattern processes a series of things one by one. Some source will initiate this process.

The request is sent along the chain implicitly and the order of the candidate objects is set, though it doesn't have to be. Each receiver contains a reference to another receiver and we can add new receivers at any time.

A good example of this pattern in the real life is the chain of events that take place in manufacturing. Each stage of the line receives the work done from the previous stage and knows to which process it passes its work. 

We want to use this pattern when we want to decouple the request's receiver and sender. It's also a good idea to use this pattern when there are multiple objects determined at runtime to be involved in the processing of a request.

To this end, specifying the handlers makes no sense, so we cannot explicitly do this without introducing issues.

Advantages to this pattern are:

* Decouples the sender from the receiver
* Simplifies your object greatly
* Gives added flexibility by distributing work amongst objects

A key concern is that the process the request goes through is obscured and somewhat difficult to debug. Debugging tools can help here, but it can still be difficult.

## Implementation

In [15]:
class Currency {
    private int amount;
    
    public Currency(int amt)
    {
        this.amount = amt;
    }
    
    public int getAmount()
    {
        return this.amount;
    }
}

In [16]:
interface DispenseChain {
    void setNextChain(DispenseChain nextChaine);
    void dispense(Currency currency);
}

In [17]:
class Pound20Dispenser implements DispenseChain {
    private DispenseChain chain;
    
    public void setNextChain(DispenseChain nextChain)
    {
        chain = nextChain;
    }
    
    public void dispense(Currency currency)
    {
        if (currency.getAmount() >= 20) {
            int num = currency.getAmount() / 20;
            int remainder = currency.getAmount() % 20;
            System.out.println("Dispensing " + num + " £20 notes");
            
            if (remainder != 0) {
                this.chain.dispense(new Currency(remainder));
            }
        } else {
            this.chain.dispense(currency);
        }
    }
}

In [18]:
class Pound10Dispenser implements DispenseChain {
    private DispenseChain chain;
    
    public void setNextChain(DispenseChain nextChain)
    {
        chain = nextChain;
    }
    
    public void dispense(Currency currency)
    {
        if (currency.getAmount() >= 10) {
            int num = currency.getAmount() / 10;
            int remainder = currency.getAmount() % 10;
            System.out.println("Dispensing " + num + " £10 notes");
            
            if (remainder != 0) {
                this.chain.dispense(new Currency(remainder));
            }
        } else {
            this.chain.dispense(currency);
        }
    }
}

In [47]:
class Pound5Dispenser implements DispenseChain {
    private DispenseChain chain;
    
    public void setNextChain(DispenseChain nextChain)
    {
        chain = nextChain;
    }
    
    public void dispense(Currency currency)
    {
        if (currency.getAmount() >= 5) {
            int num = currency.getAmount() / 5;
            int remainder = currency.getAmount() % 5;
            System.out.println("Dispensing " + num + " £5 notes");
            
            if (remainder != 0) {
                System.out.println("I only handle multiples of 5. IOU £" + remainder);
            }
        } else {
            this.chain.dispense(currency);
        }
    }
}

In [48]:
class ATM {
    public DispenseChain chain;
    
    public ATM()
    {
        this.chain = new Pound20Dispenser();
        DispenseChain chain2 = new Pound10Dispenser();
        DispenseChain chain3 = new Pound5Dispenser();
        
        this.chain.setNextChain(chain2);
        chain2.setNextChain(chain3);
    }
}

In [49]:
ATM myAtm = new ATM();

In [50]:
myAtm.chain.dispense(new Currency(78));

Dispensing 3 £20 notes
Dispensing 1 £10 notes
Dispensing 1 £5 notes
I only handle multiples of 5. IOU £3


## Examples